# DAY 3: Teacher version

Machine Learning NLP

## Imports

In [1]:
import os
import re
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import find


## Load Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# MAIN_PATH = '/content/drive/MyDrive/TP Centrale'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# !pip install py7zr --quiet

In [4]:
# import py7zr
# archive = py7zr.SevenZipFile(os.path.join(MAIN_PATH, 'datascience.stackexchange.com.7z'), mode='r')
# archive.extractall(path=os.path.join(MAIN_PATH, 'data'))
# archive.close()

In [5]:
DATA_PATH = '/content/drive/MyDrive/TP Centrale/data/'

In [6]:
# tags = pd.read_xml(DATA_PATH + 'Tags.xml')

In [7]:
posts = pd.read_xml('/content/drive/MyDrive/TP Centrale/data/Posts.xml', parser = 'etree')

In [8]:
posts = posts.loc[:1000]

In [9]:
posts = posts[~posts['Body'].isna()]

In [10]:
posts.Body.values[15]

'<p>A few gigabytes is not very "<strong>big</strong>". It\'s more like the normal size of an enterprise DB. As long as you go over PK when joining tables it should work out really well, even in the future (as long as you don\'t get TB\'s of data a day).</p>\n\n<p>Most professionals working in a big data environment consider <strong>> ~5TB</strong> as the <em>beginning</em> of the term big data. But even then it\'s not always the best way to just install the next best nosql database. You should always think about the task that you want to archive with the data (aggregate,read,search,mine,..) to find the best tools for you problem.</p>\n\n<p>i.e. if you do alot of searches in you database it would probably be better to run a solr instance/cluster and denormalize your data from a DBMS like Postgres or your SQL Server from time to time and put it into solr instead of just moving the data from sql to nosql in term of persistence and performance.</p>\n'

## Data Cleaning

In [46]:
def clean_post(post):
    # remove html, including the data inside
    # lower text 
    post = post.lower()

    pattern_html = re.compile('<.*?>')
    clean_post = pattern_html.sub('', post)

    # remove other things

    clean_post = clean_post.replace('\n', '').replace('\t', '')
    return clean_post


In [47]:
clean_post(posts.Body.values[15])

'a few gigabytes is not very "big". it\'s more like the normal size of an enterprise db. as long as you go over pk when joining tables it should work out really well, even in the future (as long as you don\'t get tb\'s of data a day).most professionals working in a big data environment consider > ~5tb as the beginning of the term big data. but even then it\'s not always the best way to just install the next best nosql database. you should always think about the task that you want to archive with the data (aggregate,read,search,mine,..) to find the best tools for you problem.i.e. if you do alot of searches in you database it would probably be better to run a solr instance/cluster and denormalize your data from a dbms like postgres or your sql server from time to time and put it into solr instead of just moving the data from sql to nosql in term of persistence and performance.'

In [48]:
posts['cleaned_body'] = posts.Body.apply(clean_post)

In [49]:
def clean_entry(entry):
    # a améliorer si besoin
    # voir si vraiment besoin aussi
    return entry.lower()

## Classic Preprocessing

The goal for this part is to implement a classic vectorization.

Hints : pay attention to stopwords, additionnal preprocessing steps and techniques of vectoriation


In [50]:
# stopwords removal (on utlise la list de sklearn car c'est de l'anglais)
# lemmatizing
# tfidf or count vectorizer
import nltk
nltk.download('punkt')
nltk.download('wordnet')

from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

vectorizer = CountVectorizer(tokenizer=LemmaTokenizer(),
                                strip_accents = 'unicode', # works 
                                stop_words = 'english', # works
                                lowercase = True, # works
                                # binary=True, # we use binary to indicate the precense of word, not the frequency (debatable)
                                )

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [51]:
vectorizer.fit(posts.cleaned_body.values)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


CountVectorizer(stop_words='english', strip_accents='unicode',
                tokenizer=<__main__.LemmaTokenizer object at 0x7f4e26988460>)

In [52]:
vectors = vectorizer.transform(posts.cleaned_body.values)

Write a function that applies the same process to the entry

In [53]:
def vectorize_entry(entry):
    cleaned_entry = clean_entry(entry)
    return vectorizer.transform([cleaned_entry])

Determine a way to use this vectorization to suggest the closest items to the entry in the database

In [54]:
# possible to think about different ways : cosine distance, sum of common words, play with the count vectorizer or the tfidf properties 

In [58]:
def vectorizer_search(entry):
    vector_entry = vectorize_entry(entry)
    dot_product = vector_entry.dot(vectors.T)
    _, indexes, values = find(dot_product) # pax vraiment la cosine distance car les vecteurs ne sont pas normalisés attention
    similar_posts_indexes = [x for _, x in sorted(zip(values, indexes), reverse=True)][:10]
    return posts.cleaned_body.values[similar_posts_indexes]



In [60]:
entry = 'This is a test'

How you can improve this approach ? 

In [ ]:
# On peut penser à le faire sur le titre, utiliser des ngrams, faire le cont vectorizer au niveau des lettres, utiliser un tfidf pour discriminer les mots importants